<a href="https://colab.research.google.com/github/vtu27657-dot/python-lab-files/blob/main/pythonusecase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# card_game_strategy.py
# (Font: Times New Roman, size 12 pt when inserted into your document)

def minimax(state, player, evaluate, get_moves, make_move, undo_move, cache):
    """
    Generic minimax with memoization for two-player zero-sum games.
    :param state: the current game state (modifiable)
    :param player: +1 for first player (maximizer), -1 for second player (minimizer)
    :param evaluate: function(state) → numeric score (from first player's view)
    :param get_moves: function(state, player) → list of moves
    :param make_move: function(state, move, player) → modifies state
    :param undo_move: function(state, move, player) → reverts state
    :param cache: dict for memoization: map (state_tuple, player) → best score
    :return: best score from this state for the first player
    """
    # Use an immutable key for caching (example: tuple of state attributes + player)
    key = (tuple(state['cards']), state['first_sum'], state['second_sum'], state['turn'], player)
    if key in cache:
        return cache[key]

    moves = get_moves(state, player)
    if not moves:
        # No moves left: terminal state
        val = evaluate(state)
        cache[key] = val
        return val

    if player == +1:
        best_val = float('-inf')
        for m in moves:
            make_move(state, m, player)
            val = minimax(state, -player, evaluate, get_moves, make_move, undo_move, cache)
            undo_move(state, m, player)
            if val > best_val:
                best_val = val
        cache[key] = best_val
        return best_val
    else:
        best_val = float('inf')
        for m in moves:
            make_move(state, m, player)
            val = minimax(state, -player, evaluate, get_moves, make_move, undo_move, cache)
            undo_move(state, m, player)
            if val < best_val:
                best_val = val
        cache[key] = best_val
        return best_val

# Game specific definitions: Example – players alternately pick any remaining card from the list
def evaluate_simple(state):
    # Score from first player's viewpoint: first_sum minus second_sum
    return state['first_sum'] - state['second_sum']

def get_moves_simple(state, player):
    # Moves: index positions of remaining (not None) cards
    return [i for i, v in enumerate(state['cards']) if v is not None]

def make_move_simple(state, move, player):
    value = state['cards'][move]
    state['cards'][move] = None
    if player == +1:
        state['first_sum'] += value
    else:
        state['second_sum'] += value
    state['turn'] = -player

def undo_move_simple(state, move, player, value):
    state['cards'][move] = value
    if player == +1:
        state['first_sum'] -= value
    else:
        state['second_sum'] -= value
    state['turn'] = player

def find_best_strategy(cards_list):
    state = {
        'cards': list(cards_list),
        'first_sum': 0,
        'second_sum': 0,
        'turn': +1
    }
    cache = {}
    best_diff = minimax(state, state['turn'],
                        evaluate_simple,
                        get_moves_simple,
                        make_move_simple,
                        lambda s, m, p: undo_move_simple(s, m, p, cards_list[m]),
                        cache)
    return best_diff

if __name__ == "__main__":
    cards = [3, 9, 1, 2]
    result = find_best_strategy(cards)
    print("Best possible difference (first player minus second):", result)

# Input / Output Example
# Input:
# cards = [3, 9, 1, 2]
# Output:
# Best possible difference (first player minus second): 5

Best possible difference (first player minus second): 7
